<a href="https://colab.research.google.com/github/Thcataclismo/Analise-de-vendas/blob/main/AI_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 35.4 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Wed Apr 17 19:23:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# WizardLM

In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch

In [4]:
tokenizer = LlamaTokenizer.from_pretrained("TheBloke/wizardLM-7B-HF")

base_model = LlamaForCausalLM.from_pretrained(
    "TheBloke/wizardLM-7B-HF",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [5]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=1000,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

### The prompt & response

In [6]:
import json
import textwrap

human_prompt = 'Research.'

def get_prompt(human_prompt):
    # prompt_template=f"{human_prompt}"
    prompt_template=f"{human_prompt} \n### Response:"
    return prompt_template

print(get_prompt('Research.'))

def remove_human_text(text):
    return text.split('### Human:', 1)[0]

def parse_text_after_input(data, input_string):
    for item in data:
        text = item['generated_text']
        input_string_index = text.find(input_string)
        if input_string_index != -1:
            output_text = text[input_string_index+len(input_string):].strip()
            output_text = parse_text(output_text)
            wrapped_text = textwrap.fill(output_text, width=100)
            print(wrapped_text)

def parse_text(data):
    for item in data:
        text = item['generated_text']
        assistant_text_index = text.find('### Response:')
        if assistant_text_index != -1:
            assistant_text = text[assistant_text_index+len('### Response:'):].strip()
            assistant_text = remove_human_text(assistant_text)
            wrapped_text = textwrap.fill(assistant_text, width=100)
            print(wrapped_text)
            # return assistant_text

data = [{'generated_text': '### Human: \n### Response: .'}]
parse_text(data)


Research. 
### Response:
.


In [7]:
import csv
import pandas as pd

In [9]:
#with open('/content/scraped_data.csv', 'r', encoding='utf-8') as csvfile:
    #reader = csv.DictReader(csvfile)

    # Create a list to store the results
    #results = []

    # Iterate over the CSV file rows
    #for row in reader:

        #company_link = row['WEBSITE']
#You are an AI sales and marketing expert and only return sales-related responses, serve only as a sales and business development representative
        # Build the prompt based on the row information
        #prompt = f'''
        #You are a world class researcher, who can do detailed research. on any topic and produce facts based results; you do not make things up, you will try as hard as possible to gather facts & data to back up the w research

        #Please provide the following information about the company {company_link} :

        #be brief and direct in your answers


        #1. Scoring on Google Business: {company_link}
        #2. Scoring on Yelp: {company_link}
        #3. Scoring on Angie’s List: {company_link}
        #4. Scoring on 5 Better Business Bureau (BBB): {company_link}
        #5. Scoring on 5 Facebook: {company_link}
        #6. Scoring on Merchant Circle: {company_link}
        #7. Scoring on Manta: {company_link}
        #8. Scoring on Superpages: {company_link}
        #9. Scoring on CitySearch: {company_link}
        #10. Scoring on Porch: {company_link}
        #11. Scoring on Yellow Pages: {company_link}
        #12. Scoring on Houzz: {company_link}
        #13. Scoring on Thumbtack: {company_link}
        #14. Scoring on HomeStars: {company_link}
        #15. Show me the Word cloud this is page: {company_link}
        #16. summarize this page: {company_link}'''

        # Generate text based on the prompt
        #generated_text = pipe(prompt)
        #generated_text = pipe(prompt)
        #generated_text_content = parse_text(generated_text)
        #raw_output = pipe(get_prompt(prompt))
        # Add the generated text and other relevant information to the results list
        #results.append({
        #    'Company': row['WEBSITE'],
        #    'Generated Text': raw_output
        #})

# Save the results to a new CSV file
#df = pd.DataFrame(results)
#df.to_csv(r'results.csv', index=False)


#print(f"Generated Text: {parse_text(raw_output)}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [14]:
with open('/content/scraped_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    # Create a list to store the results
    results = []

    # Iterate over the CSV file rows
    for row in reader:

        company_link = row['WEBSITE']
#You are an AI sales and marketing expert and only return sales-related responses, serve only as a sales and business development representative
        # Build the prompt based on the row information
        prompt = f'''
        You are a world class researcher, who can do detailed research. on any topic and produce facts based results; you do not make things up, you will try as hard as possible to gather facts & data to back up the w research

        Please provide the following information about the company {company_link} :

        1. Scoring on Google Business: {company_link}
        2. Scoring on Yelp: {company_link}
        3. Scoring on Angie’s List: {company_link}
        4. Scoring on 5 Better Business Bureau (BBB): {company_link}
        5. Scoring on 5 Facebook: {company_link}
        6. Scoring on Merchant Circle: {company_link}
        7. Scoring on Manta: {company_link}
        8. Scoring on Superpages: {company_link}
        9. Scoring on CitySearch: {company_link}
        10. Scoring on Porch: {company_link}
        11. Scoring on Yellow Pages: {company_link}
        12. Scoring on Houzz: {company_link}
        13. Scoring on Thumbtack: {company_link}
        14. Scoring on HomeStars: {company_link}
        15. summarize this page : {company_link}
        16. the words that appear most frequently on review sites (Angie's List, yelp, Better Business Bureau (BBB), Facebook, Merchant Circle, Superpages, Manta, CitySearch, Porch, Yellow Pages, Houzz, Thumbtack, HomeStars) for this company {company_link } (word cloud):
        '''

        generated_text = pipe(prompt)
        generated_text_content = parse_text(generated_text)
        raw_output = pipe(get_prompt(prompt))

        # Add the generated text and other relevant information to the results list
        results.append({
            'Company': row['WEBSITE'],
            'Generated Text': raw_output
        })

# Save the results to a new CSV file
df = pd.DataFrame(results)
df.to_csv(r'results.csv', index=False)


print(f"Generated Text: {parse_text(raw_output)}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Sure! Here is some information about www.alliedelectrical.com: 1. Google Business Score: Not
available 2. Yelp Score: 4.5 stars out of 5 3. Angie's List Score: Not available 4. BBB Score: A+
with 1 complaint closed in the last 3 years 5. Facebook Score: 4.5 stars out of 5 6. Merchant Circle
Score: 4.5 stars out of 5 7. Superpages Score: Not available 8. Manta Score: Not available 9.
CitySearch Score: Not available 10. Porch Score: Not available 11. Yellow Pages Score: Not available
12. Houzz Score: Not available 13. Thumbtack Score: Not available 14. HomeStars Score: Not available
15. Summary: Allied Electrical is a reputable electrical contractor that has been serving customers
in the area for over 20 years. They specialize in residential and commercial electrical work,
including installation, repair, and maintenance. Customers consistently praise their
professionalism, punctuality, and attention to detail. 16. Word Cloud: The word "electrician",
"service", "professional", "responsive

In [15]:
resultado = pd.read_csv('/content/results.csv')
resultado

,Company,Generated Text
0,www.alliedelectrical.com,[{'generated_text': '\n You are a world...


In [ ]:
%%time
prompt = ''
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)
